# __Neurite Orientation Dispersion and Density Imaging (NODDI) analysis notebook__
#### __Last updated on:__ 19/11/2019
#### __Author:__ Rakshit Dadarwal

### __Requirements:__
#### 1. Multi-shell DWI dataset (minimum two bvalues)
#### 2. DIPY (https://dipy.org/)
#### 2. AMICO (https://github.com/daducci/AMICO)

### __This script includes:__
#### 1. NODDI parametric maps estimation

In [ ]:
import os
import numpy as np
import nibabel as nib

import amico
amico.core.setup()

#--------------------------------------------------------------
#        Load pre-processed DWI data and brain mask 
#--------------------------------------------------------------
data_path = "/home/User/Diffusion"
file1 = 'eddy_corrected.nii.gz'
path1 = os.path.join(data_path,file1)

if os.path.exists(path1):
        
    print('Reading pre-processed diffusion data')
    img1 = nib.load(path1)
    data = img1.get_data()

    file2 = 'brain_mask.nii.gz'
    path2 = os.path.join(data_path, file2)
    img2 = nib.load(path2)
    mask = img2.get_data()

    fbval = data_path+'/dMRI.bvals'
    fbvec = data_path+'/eddy_corrected.eddy_rotated_bvecs'
    from dipy.io import read_bvals_bvecs
    bvals, bvecs = read_bvals_bvecs(fbval,fbvec)
        
    from dipy.core.gradients import gradient_table
    gtab = gradient_table(bvals, bvecs)

    #--------------------------------------------------------------
    #               Fit NODDI model 
    #--------------------------------------------------------------

    amico.util.fsl2scheme(fbval,fbvec)
    
    if not os.path.exists(data_path+'/NODDI/'):
        os.mkdir(data_path+'/NODDI')
    output_path = data_path+'/NODDI/'
    
    ae = amico.Evaluation(data_path, data_path, output_path=output_path)
      
    ae.load_data(dwi_filename = "eddy_corrected.nii.gz", scheme_filename = "dMRI.scheme", mask_filename='brain_mask.nii.gz', b0_thr = 0)
      
    ae.set_model("NODDI")
    ae.generate_kernels()
     
    ae.load_kernels()
       
    ae.fit()
    ae.save_results()
        
    print('Processing completed')
    print('Enjoy!!')
else:
    print('Diffusion data does not exist !!!')